In [ ]:
import spotipy
import pandas as pd
from spotipy.oauth2 import SpotifyOAuth

scope='user-library-read playlist-modify-public playlist-modify-private'

sp = spotipy.Spotify(auth_manager=SpotifyOAuth(client_id="6584944ee0d9495480193d9997a9efb7",
                                               client_secret="ac4205f012b64179acd46c0fbdb33f36",
                                               redirect_uri="http://127.0.0.1:5000/redirect",
                                               scope=scope))

songs = []
pl = sp.current_user_playlists()['items'];
id = pl[0]['id']
playlist = sp.playlist_tracks(id)

tracks_data = []

for track in playlist['items']:
    
    #get the track popularity
    track_uri = track['track']['uri']
    track_info = sp.track(track_uri)
    track_popularity = track_info['popularity']
  
    track_data = {
            'artist_name': track['track']['artists'][0]['name'],
            'id': track['track']['id'],
            'track_pop': track_popularity,
            'danceability': None,
            'energy': None,
            'loudness': None,
            'speechiness': None,
            'acousticness': None,
            'instrumentalness': None,
            'liveness': None,
            'valence': None,
            'tempo': None
        }
    
    audio_features = sp.audio_features([track['track']['uri']])[0]
    if audio_features:
        track_data['danceability'] = audio_features['danceability']
        track_data['energy'] = audio_features['energy']
        track_data['loudness'] = audio_features['loudness']
        track_data['speechiness'] = audio_features['speechiness']
        track_data['acousticness'] = audio_features['acousticness']
        track_data['instrumentalness'] = audio_features['instrumentalness']
        track_data['liveness'] = audio_features['liveness']
        track_data['valence'] = audio_features['valence']
        track_data['tempo'] = audio_features['tempo']
        
    tracks_data.append(track_data)

songs_df = pd.DataFrame(tracks_data)
songs_df.drop_duplicates('id')

songs_df.head()


In [ ]:
import pandas as pd
from sklearn.preprocessing import MinMaxScaler
all_songs_df = pd.read_csv(r'C:\Users\alext\OneDrive\Desktop\FYP\data\data.csv')

#rearrange columns to match the user dataframe
desired_order = ['artists', 'id', 'track_pop','danceability','energy','loudness','speechiness','acousticness','instrumentalness','liveness','valence','tempo']
all_songs_df = all_songs_df[desired_order]

#cleaning the artists column - rename and only use FIRST artist in list
all_songs_df['artists'] = all_songs_df['artists'].str.replace(r"['\[\]]", '')
all_songs_df['artists'] = all_songs_df['artists'].str.split(',').str[0]
all_songs_df = all_songs_df.rename(columns={'artists': 'artist_name'})

pop = all_songs_df[["track_pop"]].reset_index(drop = True)
scaler = MinMaxScaler()
all_songs_df['track_pop'] = pd.DataFrame(scaler.fit_transform(pop), columns = pop.columns)
all_songs_df.head()
